In [20]:
import pandas as pd

from components.model import TechnologyAdoptionModel
from components.technologies import Technologies
from components.probability import beta_with_mode_at
from data.canada import nrcan_tech_shares_df

from batch import BatchResult

historic_tech_shares = nrcan_tech_shares_df.copy()
historic_tech_shares.index = historic_tech_shares.index.swaplevel()
tech_mode_map = dict(zip(Technologies, [0.5] * len(Technologies)))

province = "Ontario"
gut = 0.33
p_mode = 0.33

h_tech_shares = historic_tech_shares.loc[province, :] / 100
att_mode_table = h_tech_shares.copy()


In [21]:


batch_parameters = {
    "N": [50],
    "province": [province],  # , "Alberta", "Ontario"],
    "random_seed": range(20, 23),
    "start_year": 2000,
    "tech_att_mode_table": [att_mode_table.copy()],
    "n_segregation_steps": [60],
    "interact": [False],
    "global_util_thresh": [gut]
}
print("price weight mode:", p_mode)
batch_parameters["price_weight_mode"] = p_mode

full_years = range(2000,2021)
scale = 2

price weight mode: 0.33


In [22]:

att_mode_tables = []
adoption_share_dfs = []
adoption_detail_dfs = []
mode_shift = 0.2
best_abs_diff = 1e12
greatest_diff_sum = None

n_fit_iterations = 3

for i in range(n_fit_iterations):
    
    b_result = BatchResult.from_parameters(batch_parameters)
    model_shares = b_result.tech_shares_df.groupby(["province","year"]).mean().drop("RunId",axis=1)
    diff = (h_tech_shares - model_shares.loc[(province,full_years),:]).loc[province,:]
    
    tech_share_abs_diff = diff.abs().sum()
    current_abs_diff = tech_share_abs_diff.sum()
    print(i, current_abs_diff)
    
    # if current is not smallest diff
    if best_abs_diff <= current_abs_diff:
        print("Performance degradation. Scaling down mode_shift")
        scale /= 2
        att_update = diff * scale
    else:
        best_abs_diff = current_abs_diff
        best_modes = att_mode_table.copy()
        att_update = diff * scale
    # print("applying cumulative update of:", att_update.abs().sum())
    att_mode_table = att_mode_table + att_update
    
    # adjust modes to where distributions are sensible
    att_mode_table[att_mode_table<0.05] = 0.05
    att_mode_table[att_mode_table>0.95] = 0.95
    
    protocol_table = att_mode_table.copy()
    protocol_table["iteration"] = i
    att_mode_tables.append(protocol_table)
    
    model_shares["iteration"] = i
    adoption_share_dfs.append(model_shares)


    print(i, diff.abs().sum())
    batch_parameters["tech_att_mode_table"] = [att_mode_table]


results_dir=WindowsPath('c:/src/canadopt/abetam/results/feature_temporal_attitudes/2cb145f260ba6716f57c3e73865f33a2') does not exist. Running model.


100%|██████████| 3/3 [00:12<00:00,  4.08s/it]


0 3.7657760607672013
0 Electric furnace                1.131700
Gas furnace                     1.616274
Heat pump                       0.554670
Oil furnace                     0.284110
Wood or wood pellets furnace    0.179021
dtype: float64
results_dir=WindowsPath('c:/src/canadopt/abetam/results/feature_temporal_attitudes/2cb145f260ba6716f57c3e73865f33a2') does not exist. Running model.


100%|██████████| 3/3 [00:11<00:00,  3.95s/it]


1 3.752442727433868
1 Electric furnace                1.125034
Gas furnace                     1.609607
Heat pump                       0.554670
Oil furnace                     0.284110
Wood or wood pellets furnace    0.179021
dtype: float64
results_dir=WindowsPath('c:/src/canadopt/abetam/results/feature_temporal_attitudes/2cb145f260ba6716f57c3e73865f33a2') does not exist. Running model.


100%|██████████| 3/3 [00:12<00:00,  4.07s/it]

2 3.405776060767201
2 Electric furnace                0.951700
Gas furnace                     1.436274
Heat pump                       0.554670
Oil furnace                     0.284110
Wood or wood pellets furnace    0.179021
dtype: float64


In [24]:
import plotly.express as px
all_attitude_modes = pd.concat(att_mode_tables)
all_attitude_modes = all_attitude_modes.melt(id_vars=["iteration"], ignore_index=False).reset_index()
px.line(all_attitude_modes, x="year", y="value", color="variable", line_dash="iteration", template="plotly")

In [25]:
adoption_shares = pd.concat(adoption_share_dfs).loc[province,:].melt(id_vars=["iteration"], ignore_index=False).reset_index()
px.line(adoption_shares, x="year", y="value", color="variable", line_dash="iteration", template="plotly")